# Chiral solutions for Type-II Dirac seesaw

In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)

## Load full solutions

In [2]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json')
ds.shape

(31211, 5)

Check that solutions are unique

In [3]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(31211, 6)

In [4]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [5]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

16805

## Check phenomenological conditions
$$ \nu+m+s=0$$

In [8]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [9]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    triplet=False
    for x in ll:
        if ll.count(x)>1:
            if ll.count(x)>2:
                triplet=True
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2 and triplet:
        MR=True
    return MR

def extract_Dirac_and_Majorana(l,s):
    ll=l.copy()
    #Check first Dirac
    for xs  in itertools.combinations(l,2):
        #print('ini',xs,ll,xs[0] in ll,xs[1] in ll)
        if (xs[0] in ll  and xs[1] in ll)  and (sum(xs)+s==0 or sum(xs)-s==0):
            ll=[x for x in ll if x in ll and x not in xs]
        #print('ll',ll)
    #Check Majorana
    for x in ll:
        if 2*x+s==0 or 2*x-s==0:
            ll.remove(x)
    return ll

def best_higgs_singlet(l):
    lenmax=np.Inf
    best_sp=0
    best_spl=l
    sps=np.unique( (np.abs( list(2*np.array(l))+[sum(x) for x in itertools.combinations(l,2) ]) ) )
    for sp in sps:
        newl=extract_Dirac_and_Majorana(l,sp)
        if len(newl)<len(best_spl):
            best_spl=newl
            best_sp=sp
    return best_sp,best_spl

def tree_level_ii(ll):
    '''
    At least a triplet and a doublet is guaranteed
    (m,m,m,...,ν,ν,...)
    '''
    fs=[]
    mrp=[]
    νrp=[]
    for x in ll:
        if ll.count(x)==1:
            fs.append(x)
        elif ll.count(x)==3:
            mrp.append(x)
        elif ll.count(x)>3:
            mrp.append(x)
            fs.append(x)
        elif ll.count(x)==2:
            νrp.append(x)
    mrp=sorted(np.unique(mrp))
    νrp=sorted(np.unique(νrp))
    fs=sorted(np.unique(fs))
    #print(len(mrp),len(νrp),ll)
    sltn=[]
    for m in mrp:
        for ν in νrp:
            #if (mrp[0]+ν in [sum(k)  for k  in itertools.combinations(fs,2)]+list(2*np.array(fs))):
            s=-m-ν# ,fs,[sum(k)  for k  in itertools.combinations(fs,2)],list(2*np.array(fs)))
            nom=mrp.copy()
            kk=nom.remove(m)
            noν=νrp.copy()
            kk=noν.remove(ν)
            massless=fs+nom+noν
            massless=extract_Dirac_and_Majorana(massless,s)
            slt={'[m,ν,s]':[m,ν,s],'massless':massless}
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)            
            sltn.append(slt)
    return sltn

#assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
#assert best_higgs_singlet([1,-11,9])==(2, [])
#assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]
#assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None

### Initialize filtered solutions

In [10]:
cl=ds.copy()
cl=cl[cl['solution'].apply(multiple_repeated)].sort_values('n').reset_index(drop=True)

In [11]:
cl

,l,k,solution,gcd,n
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6
1,"[-3, -1]","[-2, -3, 1]","[3, 3, 3, -5, -5, -7, 8]",1,7
2,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7
3,"[9, -6]","[-5, -10, 8]","[9, 9, -24, -24, -24, 25, 29]",45,7
4,"[0, 1, 2]","[-1, -9, -7, -9]","[1, 1, 1, -9, -9, 17, 23, -25]",2,8
...,...,...,...,...,...
547,"[-4, 1, 3]","[-5, -7, 6, 5]","[1, 1, 5, 11, -24, -24, -24, 25, 29]",10,9
548,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9
549,"[2, 4, 3]","[-3, -7, 0, 1]","[1, -11, 14, 14, 20, -21, -21, -21, 25]",4,9
550,"[-3, -4, 3]","[-7, 2, 0, -2]","[3, 3, 3, 7, 7, -10, -14, -31, 32]",7,9


Find solutions that satisfy the condition

In [12]:
cl['tree_level']=cl['solution'].apply(tree_level_ii)

In [13]:
cl#[cl['tree_level']].reset_index(drop=True)

,l,k,solution,gcd,n,tree_level
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[m,ν,s]': [1, -4, 3], 'massless': [5], 'sp': 10, 'sp_massless': []}]"
1,"[-3, -1]","[-2, -3, 1]","[3, 3, 3, -5, -5, -7, 8]",1,7,"[{'[m,ν,s]': [3, -5, 2], 'massless': [-7, 8], 'sp': 1, 'sp_massless': []}]"
2,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7,"[{'[m,ν,s]': [-3, 2, 1], 'massless': [1, 4], 'sp': 5, 'sp_massless': []}]"
3,"[9, -6]","[-5, -10, 8]","[9, 9, -24, -24, -24, 25, 29]",45,7,"[{'[m,ν,s]': [-24, 9, 15], 'massless': [25, 29], 'sp': 54, 'sp_massless': []}]"
4,"[0, 1, 2]","[-1, -9, -7, -9]","[1, 1, 1, -9, -9, 17, 23, -25]",2,8,"[{'[m,ν,s]': [1, -9, 8], 'massless': [23], 'sp': 46, 'sp_massless': []}]"
...,...,...,...,...,...,...
547,"[-4, 1, 3]","[-5, -7, 6, 5]","[1, 1, 5, 11, -24, -24, -24, 25, 29]",10,9,"[{'[m,ν,s]': [-24, 1, 23], 'massless': [5, 11, 25, 29], 'sp': 16, 'sp_massless': [25, 29]}]"
548,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,"[{'[m,ν,s]': [9, -10, 1], 'massless': [-5, -4, 1], 'sp': 3, 'sp_massless': [-5]}, {'[m,ν,s]': [9, 1, -10], 'massless': [-4, -10], 'sp': 14, 'sp_massless': []}]"
549,"[2, 4, 3]","[-3, -7, 0, 1]","[1, -11, 14, 14, 20, -21, -21, -21, 25]",4,9,"[{'[m,ν,s]': [-21, 14, 7], 'massless': [-11, 1, 20, 25], 'sp': 9, 'sp_massless': [1, 25]}]"
550,"[-3, -4, 3]","[-7, 2, 0, -2]","[3, 3, 3, 7, 7, -10, -14, -31, 32]",7,9,"[{'[m,ν,s]': [3, 7, -10], 'massless': [-31, -14, -10, 32], 'sp': 1, 'sp_massless': [-14, -10]}]"


Reorder the filtered solutions

In [14]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
               ).apply(len)>0].sort_values(['n','smax']).reset_index(drop=True)

cl['tree_level']=cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or 
                                                   len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
             )
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[m,ν,s]': [1, -4, 3], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5


In [15]:
cl

,l,k,solution,gcd,n,tree_level,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[m,ν,s]': [1, -4, 3], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5
1,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7,"[{'[m,ν,s]': [-3, 2, 1], 'massless': [1, 4], 'sp': 5, 'sp_massless': []}]",4
2,"[-3, -1]","[-2, -3, 1]","[3, 3, 3, -5, -5, -7, 8]",1,7,"[{'[m,ν,s]': [3, -5, 2], 'massless': [-7, 8], 'sp': 1, 'sp_massless': []}]",8
3,"[9, -6]","[-5, -10, 8]","[9, 9, -24, -24, -24, 25, 29]",45,7,"[{'[m,ν,s]': [-24, 9, 15], 'massless': [25, 29], 'sp': 54, 'sp_massless': []}]",29
4,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,8,"[{'[m,ν,s]': [3, -7, 4], 'massless': [9], 'sp': 18, 'sp_massless': []}]",9
...,...,...,...,...,...,...,...
118,"[-6, -1, -6]","[-5, -8, -5, 7]","[6, 16, 16, 20, -21, -21, -21, -25, 30]",75,9,"[{'[m,ν,s]': [-21, 16, 5], 'massless': [6, 30], 'sp': 36, 'sp_massless': []}]",30
119,"[2, -6, 2]","[-6, -3, -6, 2]","[6, 6, 6, -8, -15, -17, 26, 26, -30]",120,9,"[{'[m,ν,s]': [6, 26, -32], 'massless': [-30, -8], 'sp': 38, 'sp_massless': []}]",30
120,"[9, -1, 2]","[-1, -2, -1, -3]","[9, 9, 9, -12, -19, -19, 22, -29, 30]",8,9,"[{'[m,ν,s]': [9, -19, 10], 'massless': [-29, 30], 'sp': 1, 'sp_massless': []}]",30
121,"[-6, -9, 9]","[-9, -8, -9, -8]","[2, 21, 21, -24, -24, -24, 25, -27, 30]",81,9,"[{'[m,ν,s]': [-24, 21, 3], 'massless': [2, 25], 'sp': 27, 'sp_massless': []}]",30


### Flag equivalent solutions

In [16]:
def characterize_solution(cl,i,j=0):
    if j%2==0:
        cl.loc[i,'D_msls']=j
    else:
        cl.loc[i,'M_msls']=j
    #if j==0:
    #    cl.loc[i,'M_msls']=j
        
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    sl=[d.get('[m,ν,s]') for d in cl.loc[i,'tree_level'] if len(d.get('massless'))==j]
    if len(sl)>0:
        cl.loc[i,'nl']=0
        cl.loc[i,'nν'] =cl.loc[i,'solution'].count(sl[0][1] )
        cl.loc[i,'nr']=0
    return cl

Example: massless solutions

In [17]:
for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
    cl=characterize_solution(cl,i,0)

In [18]:
cl.dropna()

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,nν,nr
37,"[-4, 2, -3]","[-2, -5, 5, -6]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",2,9,"[{'[m,ν,s]': [-6, 9, -3], 'massless': [], 'sp': None, 'sp_massless': None}]",11,0.0,2.0,0.0,2.0,0.0
44,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,"[{'[m,ν,s]': [6, -9, 3], 'massless': [], 'sp': None, 'sp_massless': None}]",13,0.0,2.0,0.0,2.0,0.0
52,"[-2, -1, 0]","[-1, -4, -5, -4]","[1, -5, 8, 8, 8, -11, -12, -12, 15]",2,9,"[{'[m,ν,s]': [8, -12, 4], 'massless': [], 'sp': None, 'sp_massless': None}]",15,0.0,2.0,0.0,2.0,0.0
91,"[-6, -3, -6]","[-3, -7, -3, 5]","[6, 6, 6, -9, -9, 20, -22, -23, 25]",18,9,"[{'[m,ν,s]': [6, -9, 3], 'massless': [], 'sp': None, 'sp_massless': None}]",25,0.0,2.0,0.0,2.0,0.0


### Add solutions with massless fermions that get masses with some $S'$

In [19]:
for j in range(1,6):
    print(j)
    for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if d.get('sp_massless') is not None and len(d.get('sp_massless'))==0  and len(d.get('massless'))==j ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
        cl=characterize_solution(cl,i,j)

1
2
3
4
5


In [20]:
cl['D_msls']=cl['D_msls'].apply(lambda n: 10 if np.isnan(n) else n)
cl['M_msls']=cl['M_msls'].apply(lambda n: 10 if np.isnan(n) else n)
for k  in ['D_msls','M_msls','nl','nν','nr','rptd']:
    cl[k]=cl[k].astype(int)

In [21]:
cl=cl.sort_values(['n','smax','D_msls','rptd','nl','nν','nr']).reset_index(drop=True)

## Build the final table dropping out equivalent solutions 

In [35]:
fl=pd.DataFrame()

In [36]:
fl=fl.append( cl[ cl['n']==9 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [37]:
fl=fl.append( cl[ cl['n']==6 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [38]:
fl=fl.append(cl[ cl['n']==7 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [39]:
fl=fl.append(cl[ cl['n']==8 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [40]:
fl=fl.reset_index(drop=True)

In [41]:
fl

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,nν,nr,M_msls
0,"[2, -6, -7]","[-1, -9, -3, -6]","[2, 2, 2, -3, -3, 4, -5, -5, 6]",1,9,"[{'[m,ν,s]': [2, -5, 3], 'massless': [4], 'sp': 8, 'sp_massless': []}, {'[m,ν,s]': [2, -3, 1], 'massless': [6], 'sp': 12, 'sp_massless': []}]",6,10,3,0,2,0,1
1,"[-3, -1, -2]","[-1, -5, -1, -5]","[1, 2, 2, 2, 2, -5, -5, -8, 9]",1,9,"[{'[m,ν,s]': [2, -5, 3], 'massless': [-8, 9], 'sp': 1, 'sp_massless': []}]",9,2,2,0,2,0,10
2,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,"[{'[m,ν,s]': [9, 1, -10], 'massless': [-4, -10], 'sp': 14, 'sp_massless': []}]",10,2,3,0,2,0,10
3,"[-1, 0, -3]","[-2, -3, -4, -2]","[1, 4, 5, -6, -6, -6, 9, 9, -10]",1,9,"[{'[m,ν,s]': [-6, 9, -3], 'massless': [-10, 1, 4, 5], 'sp': 5, 'sp_massless': []}]",10,4,2,0,2,0,10
4,"[-4, 2, -3]","[-2, -5, 5, -6]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",2,9,"[{'[m,ν,s]': [-6, 9, -3], 'massless': [], 'sp': None, 'sp_massless': None}]",11,0,2,0,2,0,10
5,"[-5, -2, 3]","[-1, -2, 3, -4]","[3, 5, 5, -8, -8, -8, 12, 12, -13]",3,9,"[{'[m,ν,s]': [-8, 12, -4], 'massless': [-13, 3, 5], 'sp': 10, 'sp_massless': []}]",13,10,3,0,2,0,3
6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[m,ν,s]': [1, -4, 3], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5,10,2,0,2,0,1
7,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7,"[{'[m,ν,s]': [-3, 2, 1], 'massless': [1, 4], 'sp': 5, 'sp_massless': []}]",4,2,2,0,2,0,10
8,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,8,"[{'[m,ν,s]': [3, -7, 4], 'massless': [9], 'sp': 18, 'sp_massless': []}]",9,10,2,0,2,0,1
9,"[-9, 2, 3]","[-1, -9, 6, 8]","[9, -11, -11, 15, 15, 15, -16, -16]",12,8,"[{'[m,ν,s]': [15, -16, 1], 'massless': [9, -11], 'sp': 2, 'sp_massless': []}, {'[m,ν,s]': [15, -11, -4], 'massless': [9, -16], 'sp': 7, 'sp_massless': []}]",16,2,3,0,2,0,10


In [42]:
def getit(it,dk):
    it['m']=dk.get('[m,ν,s]')[0]
    it['ν']=dk.get('[m,ν,s]')[1]
    it['s']=dk.get('[m,ν,s]')[2]
    it['massless']=dk.get('massless')
    it['sp']=dk.get('sp')
    return it
    
tm=pd.DataFrame()
fl=fl.sort_values(['n','smax','D_msls','M_msls']).reset_index(drop=True)
for i in fl.index:
    it=fl.loc[i].to_dict()
    if fl.loc[i,'D_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
    elif fl.loc[i,'M_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
    else:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
        

for k in ['n','gcd','m','ν','s','D_msls','rptd','nl','nν','nr','M_msls']:
    tm[k]=tm[k].astype(int)
tm['nsmax']=tm.apply(lambda row: np.abs( row['solution'] ).max(),axis='columns' )
tm=tm.sort_values(['n','nsmax']).reset_index(drop=True)

#Delete equivalent solutions
#tm['Q']=True
#sltns={'[3, -4, 8, -9]': [3, -4, -6, -6, 7, 7, 8, -9],}
#for k in sltns.keys():
#    iq=tm[((tm['solution'].astype(str)==str(sltns[k])) &  (tm['massless'].astype(str)==k)
#          )].index[0] #unstable dark matter
#    tm.loc[iq,'Q']=False

#tm=tm[tm['Q']].reset_index(drop=True).drop('Q',axis='columns')
tm[['n','l','k','solution','gcd','m','ν','s','massless','sp','M_msls','rptd','nl','nν','nr','D_msls']
  ].sort_values(['n','M_msls','rptd','nl','nν','nr'])#.loc[[0,4]]#.loc[[0,4]]

,n,l,k,solution,gcd,m,ν,s,massless,sp,M_msls,rptd,nl,nν,nr,D_msls
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-4,3,[5],10,1,2,0,2,0,10
1,7,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,-3,2,1,"[1, 4]",5,10,2,0,2,0,2
2,8,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,3,-7,4,[9],18,1,2,0,2,0,10
4,8,"[-8, -1, -4]","[-2, -6, 4, -5]","[7, -8, -18, -18, 20, 20, 20, -23]",2,20,-18,-2,"[-23, -8, 7]",16,3,2,0,2,0,10
3,8,"[-9, 2, 3]","[-1, -9, 6, 8]","[9, -11, -11, 15, 15, 15, -16, -16]",12,15,-16,1,"[9, -11]",2,10,3,0,2,0,2
5,9,"[2, -6, -7]","[-1, -9, -3, -6]","[2, 2, 2, -3, -3, 4, -5, -5, 6]",1,2,-5,3,[4],8,1,3,0,2,0,10
10,9,"[-5, -2, 3]","[-1, -2, 3, -4]","[3, 5, 5, -8, -8, -8, 12, 12, -13]",3,-8,12,-4,"[-13, 3, 5]",10,3,3,0,2,0,10
6,9,"[-3, -1, -2]","[-1, -5, -1, -5]","[1, 2, 2, 2, 2, -5, -5, -8, 9]",1,2,-5,3,"[-8, 9]",1,10,2,0,2,0,2
8,9,"[-1, 0, -3]","[-2, -3, -4, -2]","[1, 4, 5, -6, -6, -6, 9, 9, -10]",1,-6,9,-3,"[-10, 1, 4, 5]",5,10,2,0,2,0,4
9,9,"[-4, 2, -3]","[-2, -5, 5, -6]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",2,-6,9,-3,[],None,10,2,0,2,0,0


In [48]:
kk=tm[['n','l','k','solution','gcd','m','ν','s','massless','sp','D_msls','rptd','nl','nν','nr','M_msls']].reset_index(drop=True)
kk['sltn']=kk['solution'].astype(str)
kk.sort_values('n').drop('sltn',axis='columns').reset_index(drop=True)

,n,l,k,solution,gcd,m,ν,s,massless,sp,D_msls,rptd,nl,nν,nr,M_msls
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-4,3,[5],10,10,2,0,2,0,1
1,7,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,-3,2,1,"[1, 4]",5,2,2,0,2,0,10
2,8,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,3,-7,4,[9],18,10,2,0,2,0,1
3,8,"[-9, 2, 3]","[-1, -9, 6, 8]","[9, -11, -11, 15, 15, 15, -16, -16]",12,15,-16,1,"[9, -11]",2,2,3,0,2,0,10
4,8,"[-8, -1, -4]","[-2, -6, 4, -5]","[7, -8, -18, -18, 20, 20, 20, -23]",2,20,-18,-2,"[-23, -8, 7]",16,10,2,0,2,0,3
5,9,"[2, -6, -7]","[-1, -9, -3, -6]","[2, 2, 2, -3, -3, 4, -5, -5, 6]",1,2,-5,3,[4],8,10,3,0,2,0,1
6,9,"[-3, -1, -2]","[-1, -5, -1, -5]","[1, 2, 2, 2, 2, -5, -5, -8, 9]",1,2,-5,3,"[-8, 9]",1,2,2,0,2,0,10
7,9,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,1,-10,"[-4, -10]",14,2,3,0,2,0,10
8,9,"[-1, 0, -3]","[-2, -3, -4, -2]","[1, 4, 5, -6, -6, -6, 9, 9, -10]",1,-6,9,-3,"[-10, 1, 4, 5]",5,4,2,0,2,0,10
9,9,"[-4, 2, -3]","[-2, -5, 5, -6]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",2,-6,9,-3,[],None,0,2,0,2,0,10


In [44]:
kk=tm[['n','l','k','solution','gcd','m','ν','s','massless','sp']].copy()

In [45]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [46]:
kk=tm[['n','l','k','solution','gcd','m','ν','s','massless','sp']].copy()

for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if not kk.loc[i,'massless']:
        print(i)
        kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
        #kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])

kkk=kk#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

9


In [47]:
cat solutions.tex

\begin{tabular}{llllrrrrll}
\toprule
              n &             l &                 k &                             solution & gcd &  m &   ν &   s &        massless &    sp \\
\midrule
              6 &      (-1, -2) &           (-1, 2) &                 (1, 1, 1, -4, -4, 5) &   1 &  1 &  -4 &   3 &             (5) &    10 \\
              7 &       (-1, 1) &       (-1, 0, -1) &             (1, 2, 2, -3, -3, -3, 4) &   1 & -3 &   2 &   1 &          (1, 4) &     5 \\
              8 &   (1, -3, -2) &  (-4, -9, -5, -3) &          (1, 3, 3, 3, -5, -7, -7, 9) &   1 &  3 &  -7 &   4 &             (9) &    18 \\
              8 &    (-9, 2, 3) &    (-1, -9, 6, 8) &  (9, -11, -11, 15, 15, 15, -16, -16) &  12 & 15 & -16 &   1 &        (9, -11) &     2 \\
              8 &  (-8, -1, -4) &   (-2, -6, 4, -5) &   (7, -8, -18, -18, 20, 20, 20, -23) &   2 & 20 & -18 &  -2 &    (-23, -8, 7) &    16 \\
              9 &   (2, -6, -7) &  (-1, -9, -3, -6) &      (2, 2, 2, -3, -3, 4, -5, -5, 6) &   1

In [35]:
kkk

,n,l,k,solution,gcd,m,ν,s,massless,sp
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-4,-3,[5],10
1,7,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,-3,2,-1,"[1, 4]",5
2,8,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,3,-7,-4,[9],18
3,8,"[-9, 2, 3]","[-1, -9, 6, 8]","[9, -11, -11, 15, 15, 15, -16, -16]",12,15,-16,-1,"[9, -11]",2
4,8,"[-8, -1, -4]","[-2, -6, 4, -5]","[7, -8, -18, -18, 20, 20, 20, -23]",2,20,-18,2,"[-23, -8, 7]",16
5,\boldsymbol{9},"[-4, 2, -3]","[-2, -5, 5, -6]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",2,-6,9,3,[],None


In [38]:
'LILIAM CARDEÑO ACERO'.title()

'Liliam Cardeño Acero'